In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale, scale
from tqdm import tqdm
import time
import implicit

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_hyper_pred"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_hyper_target"

In [3]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")
    
    return mrr_score / len(y_true)

In [4]:
%%time

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=2500, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in lines:
        tracks = line.strip().split(' ')
        if len(tracks) == 0:
            print(0)
        if len(tracks) == 1:
            print(1)
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 10000:
            break

CPU times: total: 5.38 s
Wall time: 5.49 s


In [5]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 453 ms
Wall time: 458 ms


# baseline

In [6]:
model_als = implicit.als.AlternatingLeastSquares(factors=200, 
                                                 regularization=10,
                                                 alpha=10,
                                                 use_native=True,
                                                 use_cg=False,
                                                 iterations=15,
                                                 random_state=42)
print('model_als.fit:')
model_als.fit(user_item)

start_time = time.time()
userid = val_id
rec_als = model_als.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_als[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_als.fit:


100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [2:11:09<00:00, 524.66s/it]




--- 5.746674060821533 seconds model_als ---
MRR@100 = 0.0201


0.020067766287411488

In [10]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                     learning_rate=0.1,
                                                     regularization=0.01,
                                                     iterations=950,
                                                     verify_negative_samples=True,
                                                     random_state=42)   
print('model_als.fit:')
model_bpr.fit(user_item)

start_time = time.time()
userid = val_id
rec_bpr = model_bpr.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_bpr[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_als.fit:


100%|███████████████████████████████████████████████| 950/950 [04:16<00:00,  3.71it/s, train_auc=99.75%, skipped=0.78%]




--- 15.113697290420532 seconds model_als ---
MRR@100 = 0.0205


0.020512453557400008

# ALS: 0.0201
# BPR: 0.0205

## Tests

In [19]:
def N_opt(model_als, model_bpr, N=100, q_bpr=1):
    
    N_als = N
    N_bpr = int(N * q_bpr)
    
    userid = val_id
        
    start_time = time.time()
    rec_als = model_als.recommend(userid, user_item_val, N=N_als, filter_already_liked_items=True)
    print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

    start_time = time.time()
    rec_bpr = model_bpr.recommend(userid, user_item_val, N=N_bpr, filter_already_liked_items=True)
    print("\n--- %s seconds model_bpr ---" % (time.time() - start_time))
    
    return rec_als, rec_bpr, N_als, N_bpr

In [20]:
def optimize(rec_als,
             rec_bpr,
             N_als,
             N_bpr,
             coef_bpr=1,
             norm_all=False,
             preproc=None):
    
    if not isinstance(norm_all, bool) or preproc not in ['minmax', 'standart', None]:
        raise ValueError('wrong params')
        
    if preproc is None:
        als_score = rec_als[1]
        bpr_score = rec_bpr[1]
    else:
        if norm_all:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_als))
                bpr_score = minmax_scale(rec_bpr[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_bpr))
            if preproc == 'standart':
                als_score = scale(rec_als[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_als))
                bpr_score = scale(rec_bpr[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_bpr))    
        else:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1], feature_range=(0, 1), axis=1)
                bpr_score = minmax_scale(rec_bpr[1], feature_range=(0, 1), axis=1)   
            if preproc == 'standart':
                als_score = scale(rec_als[1], with_mean=True, with_std=True, axis=1)
                bpr_score = scale(rec_bpr[1], with_mean=True, with_std=True, axis=1)     

    result = []           
    for i in tqdm(range(rec_als[0].shape[0])):
        als_df = pd.DataFrame({'trak_id': rec_als[0][i], 'als_score': als_score[i]})
        bpr_df = pd.DataFrame({'trak_id': rec_bpr[0][i], 'bpr_score': bpr_score[i] * coef_bpr})
        df = pd.merge(als_df, bpr_df, how="outer", on='trak_id', sort=False)
        df['als_prior'] = df.als_score.where(~df.als_score.isna(), df.bpr_score)
        df['total_score'] = np.where((df.als_score.isna()) | (df.bpr_score.isna()), df.als_prior, 
                                     (df.als_score + df.bpr_score) / 2)
        final_rec = df.sort_values(by='total_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    mrr = calc_score(target_path, pred_path)

    return mrr

In [21]:
# model_als = implicit.als.AlternatingLeastSquares(factors=200, 
#                                                  regularization=10,
#                                                  alpha=10,
#                                                  use_native=True,
#                                                  use_cg=False,
#                                                  iterations=15,
#                                                  random_state=42)
# print('model_als.fit:')
# model_als.fit(user_item)

# model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
#                                                      learning_rate=0.1,
#                                                      regularization=0.01,
#                                                      iterations=550,
#                                                      verify_negative_samples=True,
#                                                      random_state=42)   
# print('model_als.fit:')
# model_bpr.fit(user_item)

In [22]:
#rec_als, rec_bpr = N_opt(model_als, model_bpr, N=100, q_bpr=1):

In [31]:
ls = []
for N in [100, 250, 500, 750, 1000, 1500, 2000]:
    for q_bpr in [0.01, 0.1, 0.5, 1, 2, 10, 100]:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for norm_all in [True, False]:
            for preproc in [None, 'minmax', 'standart']:
                for coef_bpr in [0.005, 0.01, 0.1, 0.5, 1, 10, 100]:
                    print(f'N: {N}, q_bpr: {q_bpr}, norm_all: {norm_all}, preproc: {preproc}, coef_bpr: {coef_bpr}')
                    mrr = optimize(rec_als,
                                   rec_bpr,
                                   N_als,
                                   N_bpr,
                                   coef_bpr=coef_bpr,
                                   norm_all=norm_all,
                                   preproc=preproc)
                    ls.append((mrr, N, q_bpr, norm_all, preproc, coef_bpr))



--- 9.453793287277222 seconds model_als ---

--- 13.764113664627075 seconds model_bpr ---
N: 100, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.005


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:04<00:00, 502.29it/s]


MRR@100 = 0.0171
N: 100, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.01


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 464.69it/s]


MRR@100 = 0.0172
N: 100, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.1


...

MRR@100 = 0.0151
N: 2000, q_bpr: 100, norm_all: False, preproc: standart, coef_bpr: 10


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [01:36<00:00, 25.98it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scalin

MRR@100 = 0.0029
N: 2000, q_bpr: 100, norm_all: False, preproc: standart, coef_bpr: 100


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [01:36<00:00, 25.85it/s]


MRR@100 = 0.0024


In [32]:
data = pd.DataFrame(ls, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
0,0.017130,100,0.01,True,None,0.005
1,0.017198,100,0.01,True,None,0.010
2,0.021409,100,0.01,True,None,0.100
3,0.021613,100,0.01,True,None,0.500
4,0.021613,100,0.01,True,None,1.000
...,...,...,...,...,...,...
2053,0.020147,2000,100.00,False,standart,0.100
2054,0.020937,2000,100.00,False,standart,0.500
2055,0.015114,2000,100.00,False,standart,1.000
2056,0.002917,2000,100.00,False,standart,10.000


In [39]:
data.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
892,0.023853,750,0.01,True,minmax,0.5
884,0.023452,750,0.01,True,None,0.1
905,0.023452,750,0.01,False,None,0.1
1186,0.023445,1000,0.01,True,minmax,0.5
598,0.023299,500,0.01,True,minmax,0.5
611,0.022902,500,0.01,False,None,0.1
590,0.022902,500,0.01,True,None,0.1
1199,0.022747,1000,0.01,False,None,0.1
1178,0.022747,1000,0.01,True,None,0.1
59,0.022552,100,0.10,True,standart,0.5


In [34]:
data[data.mrr > 0.0205].sort_values(by='mrr', ascending=False)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
892,0.023853,750,0.01,True,minmax,0.5
884,0.023452,750,0.01,True,None,0.1
905,0.023452,750,0.01,False,None,0.1
1186,0.023445,1000,0.01,True,minmax,0.5
598,0.023299,500,0.01,True,minmax,0.5
...,...,...,...,...,...,...
241,0.020519,100,10.00,False,minmax,0.5
647,0.020512,500,0.10,True,standart,0.5
1278,0.020512,1000,0.50,True,standart,1.0
1181,0.020501,1000,0.01,True,None,10.0


## Add coef_mean

In [47]:
def optimize(rec_als,
             rec_bpr,
             N_als,
             N_bpr,
             coef_bpr=1,
             norm_all=False,
             preproc=None,
             coef_mean=0.5):
    
    if not isinstance(norm_all, bool) or preproc not in ['minmax', 'standart', None]:
        raise ValueError('wrong params')
        
    if preproc is None:
        als_score = rec_als[1]
        bpr_score = rec_bpr[1]
    else:
        if norm_all:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_als))
                bpr_score = minmax_scale(rec_bpr[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_bpr))
            if preproc == 'standart':
                als_score = scale(rec_als[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_als))
                bpr_score = scale(rec_bpr[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_bpr))    
        else:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1], feature_range=(0, 1), axis=1)
                bpr_score = minmax_scale(rec_bpr[1], feature_range=(0, 1), axis=1)   
            if preproc == 'standart':
                als_score = scale(rec_als[1], with_mean=True, with_std=True, axis=1)
                bpr_score = scale(rec_bpr[1], with_mean=True, with_std=True, axis=1)     

    result = []           
    for i in tqdm(range(rec_als[0].shape[0])):
        als_df = pd.DataFrame({'trak_id': rec_als[0][i], 'als_score': als_score[i]})
        bpr_df = pd.DataFrame({'trak_id': rec_bpr[0][i], 'bpr_score': bpr_score[i] * coef_bpr})
        df = pd.merge(als_df, bpr_df, how="outer", on='trak_id', sort=False)
        df['als_prior'] = df.als_score.where(~df.als_score.isna(), df.bpr_score)
        df['total_score'] = np.where((df.als_score.isna()) | (df.bpr_score.isna()), df.als_prior, 
                                     (df.als_score + df.bpr_score) * coef_mean)
        final_rec = df.sort_values(by='total_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    mrr = calc_score(target_path, pred_path)

    return mrr

In [48]:
params = data.sort_values(by='mrr', ascending=False).head()[['N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr']].values
for N, q_bpr, norm_all, preproc, coef_bpr in params:
    print(N, q_bpr, norm_all, preproc, coef_bpr)

750 0.01 True minmax 0.5
750 0.01 True None 0.1
750 0.01 False None 0.1
1000 0.01 True minmax 0.5
500 0.01 True minmax 0.5


In [51]:
ls_add_cm = []
for N in [500, 750]:
    for q_bpr in [0.01, 0.1, 1, 10]:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for norm_all in [True, False]:
            for preproc in [None, 'minmax', 'standart']:
                for coef_bpr in [0.01, 0.1, 1, 10]:
                    for coef_mean in [0.02, 0.25, 0.5, 1, 2, 10]:
                        print(f'N: {N}, q_bpr: {q_bpr}, norm_all: {norm_all}, preproc: {preproc}, coef_bpr: {coef_bpr}, coef_mean: {coef_mean}')
                        mrr = optimize(rec_als,
                                       rec_bpr,
                                       N_als,
                                       N_bpr,
                                       coef_bpr=coef_bpr,
                                       norm_all=norm_all,
                                       preproc=preproc,
                                       coef_mean=coef_mean)
                        ls_add_cm.append((mrr, N, q_bpr, norm_all, preproc, coef_bpr))



--- 6.108504772186279 seconds model_als ---

--- 12.889084577560425 seconds model_bpr ---
N: 500, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.01, coef_mean: 0.02


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 407.29it/s]


MRR@100 = 0.0172
N: 500, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.01, coef_mean: 0.25


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 444.53it/s]


MRR@100 = 0.0172
N: 500, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.01, coef_mean: 0.5


...

MRR@100 = 0.0213
N: 750, q_bpr: 10, norm_all: False, preproc: standart, coef_bpr: 10, coef_mean: 2


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:08<00:00, 290.60it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scalin

MRR@100 = 0.0200
N: 750, q_bpr: 10, norm_all: False, preproc: standart, coef_bpr: 10, coef_mean: 10


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:08<00:00, 282.24it/s]

MRR@100 = 0.0199


In [59]:
data_add_cm = pd.DataFrame(ls_add_cm, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data_add_cm['coef_mean'] = np.array([[0.02, 0.25, 0.5, 1, 2, 10] for _ in range(192)]).flatten()

In [60]:
data_add_cm

,mrr,N,q_bpr,norm_all,preproc,coef_bpr,coef_mean
0,0.017205,500,0.01,True,None,0.01,0.02
1,0.017224,500,0.01,True,None,0.01,0.25
2,0.017621,500,0.01,True,None,0.01,0.50
3,0.020339,500,0.01,True,None,0.01,1.00
4,0.020035,500,0.01,True,None,0.01,2.00
...,...,...,...,...,...,...,...
1147,0.003367,750,10.00,False,standart,10.00,0.25
1148,0.007441,750,10.00,False,standart,10.00,0.50
1149,0.021258,750,10.00,False,standart,10.00,1.00
1150,0.019973,750,10.00,False,standart,10.00,2.00


In [61]:
data_add_cm.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr,coef_mean
1017,0.023515,750,10.00,True,None,0.1,1.00
1089,0.023515,750,10.00,False,None,0.1,1.00
513,0.023467,500,10.00,False,None,0.1,1.00
441,0.023467,500,10.00,True,None,0.1,1.00
945,0.023457,750,1.00,False,None,0.1,1.00
873,0.023457,750,1.00,True,None,0.1,1.00
656,0.023452,750,0.01,False,None,0.1,0.50
584,0.023452,750,0.01,True,None,0.1,0.50
297,0.023393,500,1.00,True,None,0.1,1.00
369,0.023393,500,1.00,False,None,0.1,1.00


In [63]:
ls_add_cm = []
params = data.sort_values(by='mrr', ascending=False).head(30)[['N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr']].values
for N, q_bpr, norm_all, preproc, coef_bpr in params:
    rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
    for coef_mean in [0.5, 1, 2]:
        print(f'N: {N}, q_bpr: {q_bpr}, norm_all: {norm_all}, preproc: {preproc}, coef_bpr: {coef_bpr}, coef_mean: {coef_mean}')
        mrr = optimize(rec_als,
                       rec_bpr,
                       N_als,
                       N_bpr,
                       coef_bpr=coef_bpr,
                       norm_all=True,
                       preproc=preproc,
                       coef_mean=coef_mean)
        ls_add_cm.append((mrr, N, q_bpr, norm_all, preproc, coef_bpr, coef_mean))



--- 7.797503471374512 seconds model_als ---

--- 13.997243881225586 seconds model_bpr ---
N: 750, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.5, coef_mean: 0.5


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 376.19it/s]


MRR@100 = 0.0239
N: 750, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.5, coef_mean: 1


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 386.94it/s]


MRR@100 = 0.0211
N: 750, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.5, coef_mean: 2


...

MRR@100 = 0.0219
N: 100, q_bpr: 0.1, norm_all: True, preproc: None, coef_bpr: 0.1, coef_mean: 1


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 390.23it/s]


MRR@100 = 0.0220
N: 100, q_bpr: 0.1, norm_all: True, preproc: None, coef_bpr: 0.1, coef_mean: 2


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 395.42it/s]

MRR@100 = 0.0211


In [68]:
data_add_cm = pd.DataFrame(ls_add_cm, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr', 'coef_mean'])
data_add_cm.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr,coef_mean
0,0.023853,750,0.01,True,minmax,0.5,0.5
3,0.023452,750,0.01,True,None,0.1,0.5
6,0.023452,750,0.01,False,None,0.1,0.5
9,0.023445,1000,0.01,True,minmax,0.5,0.5
12,0.023299,500,0.01,True,minmax,0.5,0.5
18,0.022902,500,0.01,True,None,0.1,0.5
15,0.022902,500,0.01,False,None,0.1,0.5
24,0.022747,1000,0.01,True,None,0.1,0.5
21,0.022747,1000,0.01,False,None,0.1,0.5
71,0.022644,750,2.00,False,standart,1.0,2.0


In [69]:
data.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
892,0.023853,750,0.01,True,minmax,0.5
884,0.023452,750,0.01,True,None,0.1
905,0.023452,750,0.01,False,None,0.1
1186,0.023445,1000,0.01,True,minmax,0.5
598,0.023299,500,0.01,True,minmax,0.5
611,0.022902,500,0.01,False,None,0.1
590,0.022902,500,0.01,True,None,0.1
1199,0.022747,1000,0.01,False,None,0.1
1178,0.022747,1000,0.01,True,None,0.1
59,0.022552,100,0.10,True,standart,0.5


## Linspace optimization

In [73]:
def optimize(rec_als,
             rec_bpr,
             N_als,
             N_bpr,
             coef_bpr=1,
             norm_all=False,
             preproc=None):
    
    if not isinstance(norm_all, bool) or preproc not in ['minmax', 'standart', None]:
        raise ValueError('wrong params')
        
    if preproc is None:
        als_score = rec_als[1]
        bpr_score = rec_bpr[1]
    else:
        if norm_all:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_als))
                bpr_score = minmax_scale(rec_bpr[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_bpr))
            if preproc == 'standart':
                als_score = scale(rec_als[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_als))
                bpr_score = scale(rec_bpr[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_bpr))    
        else:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1], feature_range=(0, 1), axis=1)
                bpr_score = minmax_scale(rec_bpr[1], feature_range=(0, 1), axis=1)   
            if preproc == 'standart':
                als_score = scale(rec_als[1], with_mean=True, with_std=True, axis=1)
                bpr_score = scale(rec_bpr[1], with_mean=True, with_std=True, axis=1)     

    result = []           
    for i in tqdm(range(rec_als[0].shape[0])):
        als_df = pd.DataFrame({'trak_id': rec_als[0][i], 'als_score': als_score[i]})
        bpr_df = pd.DataFrame({'trak_id': rec_bpr[0][i], 'bpr_score': bpr_score[i] * coef_bpr})
        df = pd.merge(als_df, bpr_df, how="outer", on='trak_id', sort=False)
        df['als_prior'] = df.als_score.where(~df.als_score.isna(), df.bpr_score)
        df['total_score'] = np.where((df.als_score.isna()) | (df.bpr_score.isna()), df.als_prior, 
                                     (df.als_score + df.bpr_score) / 2)
        final_rec = df.sort_values(by='total_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    mrr = calc_score(target_path, pred_path)

    return mrr

In [74]:
ls_upd = []
N_arr = np.linspace(500, 1000, num=5, dtype=int)
q_bpr_arr = np.linspace(0.005, 0.02, num=5)
coef_bpr_none_arr = np.linspace(0.05, 0.2, num=5)
coef_bpr_preproc_arr = np.linspace(0.25, 1.0, num=5)

#N_arr = np.random.randint(500, high=1000, size=5, dtype=int)
# q_bpr_arr = np.random.uniform(low=0.005, high=0.02, size=5)
# coef_bpr_none_arr = np.random.uniform(low=0.05, high=0.2, size=5)
# coef_bpr_preproc_arr = np.random.uniform(low=0.25, high=1.0, size=5)
# coef_mean_arr = np.random.uniform(low=, high=, size=5)
for N in N_arr:
    for q_bpr in q_bpr_arr:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for preproc in [None, 'minmax', 'standart']:
            if preproc is None:
                coef_bpr_arr = coef_bpr_none_arr
            else:
                coef_bpr_arr = coef_bpr_preproc_arr
            for coef_bpr in coef_bpr_arr:
                print(f'N: {N}, q_bpr: {q_bpr}, norm_all: {norm_all}, preproc: {preproc}, coef_bpr: {coef_bpr}')
                mrr = optimize(rec_als,
                               rec_bpr,
                               N_als,
                               N_bpr,
                               coef_bpr=coef_bpr,
                               norm_all=True,
                               preproc=preproc)
                ls_upd.append((mrr, N, q_bpr, norm_all, preproc, coef_bpr))



--- 7.52178692817688 seconds model_als ---

--- 16.8353111743927 seconds model_bpr ---
N: 500, q_bpr: 0.005, norm_all: True, preproc: None, coef_bpr: 0.05


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 420.63it/s]


MRR@100 = 0.0211
N: 500, q_bpr: 0.005, norm_all: True, preproc: None, coef_bpr: 0.08750000000000001


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 384.21it/s]


MRR@100 = 0.0225
N: 500, q_bpr: 0.005, norm_all: True, preproc: None, coef_bpr: 0.125


...

MRR@100 = 0.0206
N: 1000, q_bpr: 0.02, norm_all: True, preproc: standart, coef_bpr: 0.8125


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 421.86it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


MRR@100 = 0.0208
N: 1000, q_bpr: 0.02, norm_all: True, preproc: standart, coef_bpr: 1.0


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 417.05it/s]

MRR@100 = 0.0211


In [76]:
N_arr 500

array([ 500,  625,  750,  875, 1000])

In [77]:
q_bpr_arr 0.01625

array([0.005  , 0.00875, 0.0125 , 0.01625, 0.02   ])

In [78]:
coef_bpr_preproc_arr 0.4375

array([0.25  , 0.4375, 0.625 , 0.8125, 1.    ])

In [79]:
data_add_cm = pd.DataFrame(ls_upd, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data_add_cm[data_add_cm.preproc == 'minmax'].sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
51,0.024801,500,0.01625,True,minmax,0.4375
111,0.024676,625,0.01250,True,minmax,0.4375
36,0.024612,500,0.01250,True,minmax,0.4375
126,0.024418,625,0.01625,True,minmax,0.4375
66,0.024398,500,0.02000,True,minmax,0.4375
171,0.024312,750,0.00875,True,minmax,0.4375
186,0.024269,750,0.01250,True,minmax,0.4375
321,0.024198,1000,0.00875,True,minmax,0.4375
52,0.024192,500,0.01625,True,minmax,0.6250
127,0.024017,625,0.01625,True,minmax,0.6250


In [82]:
ls_rand = []

N_arr = np.random.randint(425, high=575, size=10, dtype=int)
q_bpr_arr = np.random.uniform(low=0.0125, high=0.02, size=10)
coef_bpr_arr = np.random.uniform(low=0.34375, high=0.53125, size=10)

for N in N_arr:
    for q_bpr in q_bpr_arr:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for coef_bpr in coef_bpr_arr:
            print(f'N: {N}, q_bpr: {q_bpr}, norm_all: True, preproc: minmax, coef_bpr: {coef_bpr}')
            mrr = optimize(rec_als,
                           rec_bpr,
                           N_als,
                           N_bpr,
                           coef_bpr=coef_bpr,
                           norm_all=True,
                           preproc='minmax')
            ls_rand.append((mrr, N, q_bpr, True, 'minmax', coef_bpr))



--- 6.047723293304443 seconds model_als ---

--- 12.470493793487549 seconds model_bpr ---
N: 494, q_bpr: 0.014905341237382205, norm_all: True, preproc: minmax, coef_bpr: 0.4506835952174695


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 453.80it/s]


MRR@100 = 0.0244
N: 494, q_bpr: 0.014905341237382205, norm_all: True, preproc: minmax, coef_bpr: 0.353795604926389


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 414.86it/s]


MRR@100 = 0.0243
N: 494, q_bpr: 0.014905341237382205, norm_all: True, preproc: minmax, coef_bpr: 0.3700935643366595


...

MRR@100 = 0.0242
N: 546, q_bpr: 0.01704510373952061, norm_all: True, preproc: minmax, coef_bpr: 0.5215797166640299


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 494.46it/s]


MRR@100 = 0.0242
N: 546, q_bpr: 0.01704510373952061, norm_all: True, preproc: minmax, coef_bpr: 0.5017120200316704


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 496.53it/s]

MRR@100 = 0.0242


51	0.024801	500	0.01625	True	minmax	0.4375

In [84]:
data_add_cm = pd.DataFrame(ls_rand, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data_add_cm[data_add_cm.preproc == 'minmax'].sort_values(by='mrr', ascending=False).head(1)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
245,0.024815,542,0.015726,True,minmax,0.438263


## Final best

baseline 0.0205

mrr	0.024815

N 542

q_bpr 0.015726

norm_all True

preproc minmax

coef_bpr 0.438263